In [2]:
import os
import random
import metpy
import datetime
import s3fs
import numpy as np
import pandas as pd
import xarray as xr
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from pyresample import geometry, grid
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from skimage.filters import laplace
from skimage.filters import unsharp_mask
from skimage.transform import resize
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

fs = s3fs.S3FileSystem(anon = True)
model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")
random.seed(42)

2022-12-30 14:26:36.219030: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-12-30 14:26:36.226877: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:83:00.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2022-12-30 14:26:36.227848: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 1 with properties: 
pciBusID: 0000:84:00.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2022-12-30 14:26:36.227973: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-12-30 14:26:36.230574: I tensorflow/stream_executor/platform/default/d

In [2]:
def control(img):
    """
    Returns a dictionary of errors for the given file without sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to evaluate the error of
    Precondition: file is a string
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    control = resize(smallimg, (height, width))
    mae = mean_absolute_error(img, control)
    rmse = mean_squared_error(img, control, squared=False)
    return {'Control_MAE': mae, 'Control_RMSE': rmse}

def laplace_sharpening(img):
    """
    Returns a dictionary of errors for the given file using laplace sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    laplace_edges = laplace(blurryimg)
    sharpimg = blurryimg + laplace_edges
    mae = mean_absolute_error(img, sharpimg)
    rmse = mean_squared_error(img, sharpimg, squared=False)
    return {'LP_MAE': mae, 'LP_RMSE': rmse}

def unsharpmask_sharpening(img):
    """
    Returns a dictionary of errors for the given file using unsharpmask sharpening. To normalize images,
    each image is divided by its maximum value, and then multiplied by the same value after sharpening
    is completed.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a str
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    sharpimg = unsharp_mask(blurryimg/blurryimg.max(), radius=1, amount=1)*blurryimg.max()
    mae = mean_absolute_error(img, sharpimg)
    rmse = mean_squared_error(img, sharpimg, squared=False)
    return {'UM_MAE': mae, 'UM_RMSE': rmse}

def find_npy_file(path, file, band):
    """
    Returns the string name of another file from the same time for a given GOES band npy file,
    or None if there is no such file in the given directory.
    
    Parameter path: The path where the returned file is located in
    Precondition: path is a string to a directory relative to the current one, with .npy files
    
    Parameter file: A file from the same time the returned file should have
    Precondition: file is a string with the name of the original file, and includes the path
    
    Parameter band: GOES band the returned file should be from
    Precondition: band is a string of length 2
    """
    for x in os.listdir(path):
        if x[19:21] == band and file[23:37]==x[27:41]:
            return x

def update_dict(img, data):
    """
    Updates the dictionary with sharpening errors of a given file.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    
    Parameter data: Dictionary containing sharpening errors
    Precondition: data is a dict
    """
    newdata = {}
    newdata.update(control(img))
    newdata.update(laplace_sharpening(img))
    newdata.update(unsharpmask_sharpening(img))
    newdata.update(esrgan_sharpening(img))
    data.update(newdata)

In [3]:
def load_image(array):
    """
    Returns loaded .npy file.
    
    Parameter path: Path to load .npy file from
    Precondition: path is a string
    """
    result = np.zeros((array.shape[0], array.shape[1], 3))
    result[:,:,0]= array
    result[:,:,1]= array
    result[:,:,2]= array
#     result = result.astype('uint8')
    return result

def preprocess_image(array):
    """
    Returns preprocessed input array.
    
    Parameter array: array to preprocess
    Precondition: array is a numpy array
    """
    hr_image = array
    hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 2) * 2
    hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
    hr_image = tf.cast(hr_image, tf.float32)
    return tf.expand_dims(hr_image, 0)

def downscale_image(image):
    """
    Returns low resolution image after scaling down input image using nearest neighbor downsampling.

    Parameter image: 3D of 4D tensor of preprocessed image
    Precondition: image is a tensor
    """
#     image_size = []
#     if len(image.shape) == 3:
#         image_size = [image.shape[1], image.shape[0]]
#     else:
#         raise ValueError("Dimension mismatch. Can work only on single image.")

#     image = tf.squeeze(tf.cast(image, tf.uint8))

#     lr_image = np.asarray(Image.fromarray(image.numpy()).resize([image_size[0] // 2, image_size[1] // 2]))
#     lr_image = tf.expand_dims(lr_image, 0)
#     lr_image = tf.cast(lr_image, tf.float32)
#     return lr_image
    height, width = image.shape
    result = np.zeros((image.shape[0]//2, image.shape[1]//2, 3))
    smallimg = resize(image, (round(height/2), round(width/2)))
    result[:,:,0] = smallimg
    result[:,:,1] = smallimg
    result[:,:,2] = smallimg
    lr_image = tf.expand_dims(result, 0)
    lr_image = tf.cast(lr_image, tf.float32)
    return lr_image

def esrgan_sharpening(image):
    """
    Returns dictionary containing errors after using the ESRGAN model.
    
    Parameter path: path of image not including file name
    Precondition: path is a string
    
    Parameter file: file of the image
    Precondition: file is a string of a .npy file
    """
#     hr_image = preprocess_image(load_image(image))
# #     lr_image = downscale_image(tf.squeeze(hr_image))
#     lr_image = downscale_image(image)
#     fake_image = model(lr_image)
#     fake_image = tf.squeeze(fake_image)
    
#     hr_image = tf.squeeze(hr_image).numpy()[:,:,0]
#     lr_image = tf.squeeze(tf.cast(lr_image, tf.uint8))
#     lr_image = np.asarray(Image.fromarray(tf.squeeze(lr_image).numpy()).resize([hr_image.shape[0], hr_image.shape[1]]))[:,:,0]
#     fake_image = resize(fake_image.numpy()[:,:,0], [hr_image.shape[0], hr_image.shape[1]])
    
#     control_mae = mean_absolute_error(hr_image, lr_image)
#     control_rmse = mean_squared_error(hr_image, lr_image, squared=False)

#     mae = mean_absolute_error(hr_image, fake_image)
#     rmse = mean_squared_error(hr_image, fake_image, squared=False)

    hr_image = preprocess_image(load_image(image))
    lr_image = downscale_image(image)
    fake_image = model(lr_image)
    fake_image = tf.squeeze(fake_image)

    hr_image = tf.squeeze(hr_image).numpy()
    lr_image = tf.squeeze(lr_image).numpy()
    lr_image = resize(lr_image, (hr_image.shape[0], hr_image.shape[1], 3)).ravel()
    fake_image = resize(fake_image.numpy(), (hr_image.shape[0], hr_image.shape[1], 3)).ravel()
    hr_image = hr_image.ravel()

    control_mae = mean_absolute_error(hr_image, lr_image)
    control_rmse = mean_squared_error(hr_image, lr_image, squared=False)

    mae = mean_absolute_error(hr_image, fake_image)
    rmse = mean_squared_error(hr_image, fake_image, squared=False)
    return {'Control_MAE*': control_mae, 'Control_RMSE*': control_rmse, 'SR_MAE': mae, 'SR_RMSE': rmse}

# Error

In [4]:
mean = []
std = []
path = '../../GOES_Files/npy_files/'
pathCM = '../../GOES_Files/clear_sky_mask/'
files = os.listdir(pathCM)

for n in range(5):
    # data = []
    dataCM = []
    x = 0
    while x < 10:
        CM = random.choice(files)
        files.remove(CM)
        try:
            file07 = None
            file14 = None
            while file07 == None or file14 == None:
                file07 = find_npy_file(path, CM, '07')
                file14 = find_npy_file(path, CM, '14')

#             data07 = {'File': file07, 'Band': file07[19:21]}
            img07 = np.load(path + file07)
#             update_dict(img07, data07)
#             data14 = {'File': file14, 'Band': file14[19:21]}
            img14 = np.load(path + file14)
#             update_dict(img14, data14)

#             datadiff = {'Band': 'diff'}
#             diffimg = img07 - img14
#             update_dict(diffimg, datadiff)

#             data.append(data07)
#             data.append(data14)
#             data.append(datadiff)

            mask = np.load(pathCM + CM)
            mask = mask.astype(bool)
            img07[mask]=0
            img14[mask]=0

            data07CM = {'File': file07, 'Band': file07[19:21]}
            update_dict(img07, data07CM)
            data14CM = {'File': file14, 'Band': file14[19:21]}
            update_dict(img14, data14CM)

            datadiffCM = {'Band': 'diff'}
            diffimgCM = img07 - img14
            update_dict(diffimgCM, datadiffCM)

            dataCM.append(data07CM)
            dataCM.append(data14CM)
            dataCM.append(datadiffCM)
            
            x += 1
        except:
            print(CM)
    # df = pd.DataFrame(data)
    # mean.append(df.groupby(df['Band']).agg({'Control_MAE': 'mean', 'Control_RMSE': 'mean', 'Control_MAE*': 'mean', 'Control_RMSE*': 'mean',
    #                             'LP_MAE': 'mean', 'LP_RMSE': 'mean', 'UM_MAE': 'mean', 'UM_RMSE': 'mean', 'SR_MAE': 'mean', 'SR_RMSE': 'mean'}))
    dfCM = pd.DataFrame(dataCM)
    mean.append(dfCM.groupby(dfCM['Band']).agg({'Control_MAE': 'mean', 'Control_RMSE': 'mean', 'Control_MAE*': 'mean', 'Control_RMSE*': 'mean',
                                'LP_MAE': 'mean', 'LP_RMSE': 'mean', 'UM_MAE': 'mean', 'UM_RMSE': 'mean', 'SR_MAE': 'mean', 'SR_RMSE': 'mean'}))
    # std.append(df.groupby(df['Band']).agg({'Control_MAE': 'std', 'Control_RMSE': 'std', 'Control_MAE*': 'std', 'Control_RMSE*': 'std',
    #                             'LP_MAE': 'std', 'LP_RMSE': 'std', 'UM_MAE': 'std', 'UM_RMSE': 'std', 'SR_MAE': 'std', 'SR_RMSE': 'std'}))
    std.append(dfCM.groupby(dfCM['Band']).agg({'Control_MAE': 'std', 'Control_RMSE': 'std', 'Control_MAE*': 'std', 'Control_RMSE*': 'std',
                            'LP_MAE': 'std', 'LP_RMSE': 'std', 'UM_MAE': 'std', 'UM_RMSE': 'std', 'SR_MAE': 'std', 'SR_RMSE': 'std'}))

In [5]:
means = pd.DataFrame()
stds = pd.DataFrame()

for m in mean:
    means = pd.concat((m, means))
for s in std:
    stds = pd.concat((s, stds))

In [6]:
print('Average Mean Error')
means.groupby(means.index).agg({'Control_MAE': 'mean', 'Control_RMSE': 'mean', 'Control_MAE*': 'mean', 'Control_RMSE*': 'mean',
                                'LP_MAE': 'mean', 'LP_RMSE': 'mean', 'UM_MAE': 'mean', 'UM_RMSE': 'mean', 'SR_MAE': 'mean', 'SR_RMSE': 'mean'})

Average Mean Error


,Control_MAE,Control_RMSE,Control_MAE*,Control_RMSE*,LP_MAE,LP_RMSE,UM_MAE,UM_RMSE,SR_MAE,SR_RMSE
Band,,,,,,,,,,
07,13.915643,36.760688,13.915643,38.435520,12.054020,33.253745,12.098536,34.078529,9.910770,30.159314
14,13.765181,36.409695,13.765182,38.066411,11.925032,32.934256,11.962927,33.749301,9.816120,29.860016
diff,0.201373,0.449459,0.201373,0.489312,0.174699,0.405447,0.180083,0.416131,0.353616,0.577475


In [8]:
print("Standard Deviation of Means of Error")
stds.groupby(stds.index).agg({'Control_MAE': 'std', 'Control_RMSE': 'std', 'Control_MAE*': 'std', 'Control_RMSE*': 'std',
                                'LP_MAE': 'std', 'LP_RMSE': 'std', 'UM_MAE': 'std', 'UM_RMSE': 'std', 'SR_MAE': 'std', 'SR_RMSE': 'std'})

Standard Deviation of Means of Error


,Control_MAE,Control_RMSE,Control_MAE*,Control_RMSE*,LP_MAE,LP_RMSE,UM_MAE,UM_RMSE,SR_MAE,SR_RMSE
Band,,,,,,,,,,
07,0.795941,1.997958,0.795940,1.772027,0.676456,1.758007,0.696615,1.815151,0.571540,1.387425
14,0.787309,1.964069,0.787309,1.744017,0.668284,1.727073,0.688821,1.783268,0.562235,1.361227
diff,0.022960,0.051159,0.022960,0.052988,0.020343,0.046463,0.021063,0.047800,0.023132,0.049268


In [ ]:
print("Average Standard Deviation of Error")
stds.groupby(stds.index).agg({'Control_MAE': 'mean', 'Control_RMSE': 'mean', 'Control_MAE*': 'mean', 'Control_RMSE*': 'mean',
                                'LP_MAE': 'mean', 'LP_RMSE': 'mean', 'UM_MAE': 'mean', 'UM_RMSE': 'mean', 'SR_MAE': 'mean', 'SR_RMSE': 'mean'})